In [4]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

ImportError: cannot import name 'is_internal_type'

In [4]:
time_intervals = [(x, x+3) for x in range(0, 24) if x%3 == 0]
time_intervals[-1] = (21, 0)
print(time_intervals)

[(0, 3), (3, 6), (6, 9), (9, 12), (12, 15), (15, 18), (18, 21), (21, 0)]


In [5]:
weekday_intervals = list(range(1, 8))
print(weekday_intervals)

[1, 2, 3, 4, 5, 6, 7]


In [6]:
accuracy_intervals = [(0, 18), (19, 50), (51, 60), (61, 70), (71, 100), (101, 100000)]
print(accuracy_intervals)

[(0, 18), (19, 50), (51, 60), (61, 70), (71, 100), (101, 100000)]


In [3]:
def get_all_combinations(time_ints, weekday_ints, accuracy_ints):
    ans_dicts = []
    for t in time_ints:
        for w in weekday_ints:
            for a in accuracy_ints:
                ans_dicts.append({'time_ints': t, 'weekday_ints': w, 'accuracy_ints' :a})
    return ans_dicts

In [7]:
features_grid = get_all_combinations(time_intervals, weekday_intervals, accuracy_intervals)
#pd.DataFrame(get_all_combinations(time_intervals, weekday_intervals, accuracy_intervals))

In [8]:
features_grid

[{'accuracy_ints': (0, 18), 'time_ints': (0, 3), 'weekday_ints': 1},
 {'accuracy_ints': (19, 50), 'time_ints': (0, 3), 'weekday_ints': 1},
 {'accuracy_ints': (51, 60), 'time_ints': (0, 3), 'weekday_ints': 1},
 {'accuracy_ints': (61, 70), 'time_ints': (0, 3), 'weekday_ints': 1},
 {'accuracy_ints': (71, 100), 'time_ints': (0, 3), 'weekday_ints': 1},
 {'accuracy_ints': (101, 100000), 'time_ints': (0, 3), 'weekday_ints': 1},
 {'accuracy_ints': (0, 18), 'time_ints': (0, 3), 'weekday_ints': 2},
 {'accuracy_ints': (19, 50), 'time_ints': (0, 3), 'weekday_ints': 2},
 {'accuracy_ints': (51, 60), 'time_ints': (0, 3), 'weekday_ints': 2},
 {'accuracy_ints': (61, 70), 'time_ints': (0, 3), 'weekday_ints': 2},
 {'accuracy_ints': (71, 100), 'time_ints': (0, 3), 'weekday_ints': 2},
 {'accuracy_ints': (101, 100000), 'time_ints': (0, 3), 'weekday_ints': 2},
 {'accuracy_ints': (0, 18), 'time_ints': (0, 3), 'weekday_ints': 3},
 {'accuracy_ints': (19, 50), 'time_ints': (0, 3), 'weekday_ints': 3},
 {'accuracy

In [ ]:
def fgeneration_data(df):
    mintue = df['time']%60
    df['hour'] = df['time']//60
    df['weekday'] = df['hour']//24
    df['hour'] = ((df['hour']%24+1)+mintue/60.0)
    df['weekday'] = int(df['weekday']%7+1)
    df.drop(['time'], axis=1, inplace=True)
    dfo = df
    return dfo

def preprocessing_data(df, fw):
    df.drop(['accuracy'], axis=1, inplace=True)
    df['hour'] = df['hour']*fw[2]
    df['weekday'] = df['weekday']*fw[3]
    dfo = df
    return dfo

# Classification inside one grid cell.
def process_one_cell(df_train, df_test, grid_condition, fw):
    # {'accuracy_ints': (0, 18), 'time_ints': (0, 3), 'weekday_ints': 4} = grid_condition
    accuracy_min, accuracy_max = grid_condition['accuracy_ints']
    hh_min, hh_max = grid_condition['time_ints']
    weekday_v = int(grid_condition['weekday_ints'])
    #Working on df_train
    filter_df = lambda df: df[(df['accuracy'] >= accuracy_min) & (df['accuracy'] <= accuracy_max) &
                               (df['hour'] >= hh_min) & (df['hour'] < hh_max) &
                                (df['weekday'] >= weekday_v))]
    df_cell_train = filter_df(df_train)
    df_cell_train = preprocessing_data(df_cell_train, fw)
    
    place_counts = df_cell_train.place_id.value_counts()
    mask = (place_counts[df_cell_train.place_id.values] >= th).values
    df_cell_train = df_cell_train.loc[mask]
    
    #Working on df_test
    # to be delete: df_cell_test = df_test.loc[df_test.grid_cell == grid_id]
    df_cell_test = filter_df(df_test)
    df_cell_test = preprocessing_data(df_cell_test, fw)
    row_ids = df_cell_test.index

    #Feature engineering on x and y
    df_cell_train.loc[:,'x'] *= fw[0]
    df_cell_train.loc[:,'y'] *= fw[1]
    df_cell_test.loc[:,'x'] *= fw[0]
    df_cell_test.loc[:,'y'] *= fw[1]
    
    #Preparing data
    le = LabelEncoder()
    y = le.fit_transform(df_cell_train.place_id.values)
    X = df_cell_train.drop(['place_id'], axis=1).values.astype(float)
    X_test = df_cell_test.values.astype(float)

    #Applying the classifier
    clf = KNeighborsClassifier(n_neighbors=26, weights='distance', 
                               metric='manhattan', n_jobs=2)
    clf.fit(X, y)
    y_pred = clf.predict_proba(X_test)
    pred_labels = le.inverse_transform(np.argsort(y_pred, axis=1)[:,::-1][:,:3]) 
    
    return pred_labels, row_ids
   

def process_grid(df_train, df_test, features_grid, fw):
    """
    Iterates over all grid cells, aggregates the results and makes the
    submission.
    """ 
    preds = np.zeros((df_test.shape[0], 3), dtype=np.int64)
    for it in features_grid:
        #Applying classifier to one grid cell
        #TODO: add preparation
        pred_labels, row_ids = process_one_cell(fgeneration_data(df_train), fgeneration_data(df_test), it, fw)
        #Updating predictions
        preds[row_ids] = pred_labels
        print(it)
        #print("Row %d/%d elapsed time: %.2f seconds" % (i+1, (int)(size/x_step),(time.time() - start_time_row)))

    print('Generating submission file ...')
    #Auxiliary dataframe with the 3 best predictions for each sample
    df_aux = pd.DataFrame(preds, dtype=str, columns=['l1', 'l2', 'l3'])  
    
    #Concatenating the 3 predictions for each sample
    ds_sub = df_aux.l1.str.cat([df_aux.l2, df_aux.l3], sep=' ')
    
    #Writting to csv
    ds_sub.name = 'place_id'
    ds_sub.to_csv('sub_knn.csv', index=True, header=True, index_label='row_id')  

In [ ]:
fw = [500., 1000., 4., 3., 2., 8.] #feature weights (black magic here)
th = 5 #Keeping place_ids with more than th samples.

In [ ]:
print('Loading data ...')

df_train = pd.read_csv('../input/train.csv',
                       usecols=['row_id','x','y', 'accuracy', 'time','place_id'], 
                       index_col = 0)
df_test = pd.read_csv('../input/test.csv',
                      usecols=['row_id','x','y', 'accuracy' ,'time'],
                      index_col = 0)

print('Preparing train data')
#Feature engineering

#Solving classification problems inside each grid cell 
process_grid(df_train, df_test, features_grid, fw)

485300.0

In [11]:
(485300%24+1)+20/60

21.333333333333332

In [14]:
int((485300//24)%7+1)

5